In [1]:
import tensorflow as tf
K = tf.keras.backend
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names


In [11]:
data=pd.read_csv('../data/data_bert.csv',encoding='utf-8')

In [12]:
data.head()

,Unnamed: 0,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,...,tag_bert758,tag_bert759,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767
0,0,207000097,24696418,1.0,5,0,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
1,1,207000097,32507861,1.0,6,2,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
2,2,207000097,39060478,1.0,6,2,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
3,3,207000097,85909646,1.0,6,2,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
4,4,207000097,172874760,1.0,5,0,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011


In [4]:
data.tail()

,Unnamed: 0,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,...,tag_bert758,tag_bert759,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767
845459,845459,507078823,9873156,0.440,6,0,0,12,0.999862,0.999886,...,0.974894,0.995783,-0.989222,0.999992,0.94585,-0.997905,-0.995735,-0.999130,-0.994474,0.993043
845460,845460,764155292,85274975,0.232,5,0,0,19,0.999687,0.999936,...,0.999685,0.996559,-0.993992,0.999981,0.94837,-0.999431,-0.997548,-0.999439,-0.977021,0.855312
845461,845461,764155292,51463228,0.000,6,0,0,19,0.999687,0.999936,...,0.999685,0.996559,-0.993992,0.999981,0.94837,-0.999431,-0.997548,-0.999439,-0.977021,0.855312
845462,845462,764155292,693496460,0.005,3,0,0,19,0.999687,0.999936,...,0.999685,0.996559,-0.993992,0.999981,0.94837,-0.999431,-0.997548,-0.999439,-0.977021,0.855312
845463,845463,718667726,346747239,0.753,4,0,1,57,0.998748,0.999991,...,0.980760,0.997610,-0.997252,0.999985,0.96784,-0.996740,-0.999278,-0.996762,-0.996212,0.990695


In [13]:
data=data.drop(['Unnamed: 0'],axis=1)

In [14]:
data['user_nm']=0.
data['dance_nm']=0.

In [15]:
from tqdm import tqdm

In [16]:
#user_nm赋值
k = 10000/(data["user_id"].max()-data["user_id"].min())
for i in tqdm(range(len(data))):
    
    transform_value=k*(data["user_id"][i]-data["user_id"].min()) 
    data["user_nm"][i]=transform_value


  0%|                                                                                       | 0/845464 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_10452\3739655352.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["user_nm"][i]=transform_value
100%|████████████████████████████████████████████████████████████████████████| 845464/845464 [2:53:23<00:00, 81.27it/s]


In [17]:
#dance_nm赋值
k1 = 10000/(data["dance_id"].max()-data["dance_id"].min())
for i in tqdm(range(len(data))):
    transform_value=k1*(data["dance_id"][i]-data["dance_id"].min())+1 
#     print(transform_value)
    data["dance_nm"][i]=transform_value

  0%|                                                                                       | 0/845464 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_10452\3916622010.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["dance_nm"][i]=transform_value
100%|████████████████████████████████████████████████████████████████████████| 845464/845464 [2:56:02<00:00, 80.04it/s]


In [19]:
data.to_csv("data_bert_nm.csv",encoding='utf-8')

In [ ]:
cols = data.columns

In [ ]:
cols

In [7]:
len([column for column in data][7:])

1536

In [3]:
data=pd.read_csv("../data/data_bert_nm.csv",encoding='utf-8')

In [4]:
data.head()

,Unnamed: 0,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,...,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767,user_nm,dance_nm
0,0,207000097,24696418,1.0,5,0,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,115.001742,1.0
1,1,207000097,32507861,1.0,6,2,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,151.376687,1.0
2,2,207000097,39060478,1.0,6,2,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,181.889753,1.0
3,3,207000097,85909646,1.0,6,2,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,400.048661,1.0
4,4,207000097,172874760,1.0,5,0,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,805.012401,1.0


In [5]:
data=data.drop(['Unnamed: 0'],axis=1)

In [6]:
data

,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,title_bert2,...,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767,user_nm,dance_nm
0,207000097,24696418,1.000,5,0,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,115.001742,1.000000
1,207000097,32507861,1.000,6,2,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,151.376687,1.000000
2,207000097,39060478,1.000,6,2,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,181.889753,1.000000
3,207000097,85909646,1.000,6,2,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,400.048661,1.000000
4,207000097,172874760,1.000,5,0,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,805.012401,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845459,507078823,9873156,0.440,6,0,0,12,0.999862,0.999886,0.999005,...,-0.989222,0.999992,0.945850,-0.997905,-0.995735,-0.999130,-0.994474,0.993043,45.975401,3893.739553
845460,764155292,85274975,0.232,5,0,0,19,0.999687,0.999936,0.999992,...,-0.993992,0.999981,0.948370,-0.999431,-0.997548,-0.999439,-0.977021,0.855312,397.093237,7228.636873
845461,764155292,51463228,0.000,6,0,0,19,0.999687,0.999936,0.999992,...,-0.993992,0.999981,0.948370,-0.999431,-0.997548,-0.999439,-0.977021,0.855312,239.644682,7228.636873
845462,764155292,693496460,0.005,3,0,0,19,0.999687,0.999936,0.999992,...,-0.993992,0.999981,0.948370,-0.999431,-0.997548,-0.999439,-0.977021,0.855312,3229.351326,7228.636873


In [7]:
#划分特征
sparse_cols= ['dance_nm','user_nm','level','sex_em','favorites','play']
dense_cols= [column for column in data][7:-2]

In [7]:
len(dense_cols)

1536

In [8]:
for feat in sparse_cols:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [9]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1,embedding_dim=4)
                       for i,feat in enumerate(sparse_cols)] + [DenseFeat(feat, 1,)
                      for feat in dense_cols]

In [10]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
#数据构建
train_data = data.sample(frac=1)
train_data.head()

In [11]:
train, test = train_test_split(data, test_size=0.2,random_state=2022)

In [12]:
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [13]:
#搭建w&d模型
def wd_model(sparse_columns, dense_columns, train, test):

    ####### sparse features ##########
    sparse_input = []
    lr_embedding = []
    for col in sparse_columns:
        ## lr_embedding
        _input = Input(shape=(1,))
        sparse_input.append(_input)

        nums = pd.concat((train[col], test[col])).nunique() + 1
#         print(nums)
        embed = Flatten()(Embedding(nums, 4, input_length=1, embeddings_regularizer=tf.keras.regularizers.l2(0.5))(_input))  # shape=(None, 1, 4) -> shape=(None, 4)

        lr_embedding.append(embed)

    fst_order_sparse_layer = concatenate(lr_embedding)

    ####### dense features ##########
    dense_input = []
    for col in dense_columns:
        _input = Input(shape=(1,))
        dense_input.append(_input)
    concat_dense_input = concatenate(dense_input) # (None, 1)  (None, 13)
    fst_order_dense_layer = Dense(4, activation='relu')(concat_dense_input)

    ####### linear concat ##########
    linear_part = concatenate([fst_order_dense_layer, fst_order_sparse_layer]) ## (None, 4) (None, 26*4) -> (None, 4+26*4)

    #######dnn layer##########
    fc_layer = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(128)(fst_order_dense_layer))))
    fc_layer = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(64)(fc_layer))))
    fc_layer = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(64)(fc_layer))))
    fc_layer = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(32)(fc_layer))))

    ######## output layer ##########
    output_layer =  concatenate([linear_part, fc_layer])  # (None, 4+26*4 + 32)
    output_layer = Dense(1, activation='sigmoid')(output_layer)

    model = Model(inputs=sparse_input+dense_input, outputs=output_layer)

    return model
model = wd_model(linear_feature_columns,dnn_feature_columns)
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

In [14]:
history = model.fit(train_model_input, train[['lable']].values,
                    batch_size=1024, epochs=100, verbose=1, validation_split=0.2)

Epoch 1/100
529/529 [==============================] - 87s 151ms/step - loss: 0.4844 - binary_crossentropy: 0.4816 - val_loss: 0.4426 - val_binary_crossentropy: 0.4373
Epoch 2/100
529/529 [==============================] - 80s 152ms/step - loss: 0.4281 - binary_crossentropy: 0.4196 - val_loss: 0.4500 - val_binary_crossentropy: 0.4398
Epoch 3/100
529/529 [==============================] - 81s 153ms/step - loss: 0.4057 - binary_crossentropy: 0.3924 - val_loss: 0.4813 - val_binary_crossentropy: 0.4665
Epoch 4/100
529/529 [==============================] - 79s 150ms/step - loss: 0.3804 - binary_crossentropy: 0.3636 - val_loss: 0.5257 - val_binary_crossentropy: 0.5078
Epoch 5/100
529/529 [==============================] - 79s 149ms/step - loss: 0.3589 - binary_crossentropy: 0.3401 - val_loss: 0.5617 - val_binary_crossentropy: 0.5422
Epoch 6/100
529/529 [==============================] - 80s 151ms/step - loss: 0.3426 - binary_crossentropy: 0.3229 - val_loss: 0.5885 - val_binary_crossentropy:

529/529 [==============================] - 80s 152ms/step - loss: 0.2570 - binary_crossentropy: 0.2245 - val_loss: 0.9127 - val_binary_crossentropy: 0.8799
Epoch 50/100
529/529 [==============================] - 80s 152ms/step - loss: 0.2565 - binary_crossentropy: 0.2240 - val_loss: 0.8804 - val_binary_crossentropy: 0.8476
Epoch 51/100
529/529 [==============================] - 80s 150ms/step - loss: 0.2565 - binary_crossentropy: 0.2240 - val_loss: 0.8797 - val_binary_crossentropy: 0.8469
Epoch 52/100
529/529 [==============================] - 80s 151ms/step - loss: 0.2551 - binary_crossentropy: 0.2225 - val_loss: 0.8950 - val_binary_crossentropy: 0.8622
Epoch 53/100
529/529 [==============================] - 79s 150ms/step - loss: 0.2549 - binary_crossentropy: 0.2224 - val_loss: 0.9280 - val_binary_crossentropy: 0.8951
Epoch 54/100
529/529 [==============================] - 80s 151ms/step - loss: 0.2546 - binary_crossentropy: 0.2220 - val_loss: 0.8837 - val_binary_crossentropy: 0.8508

529/529 [==============================] - 81s 153ms/step - loss: 0.2103 - binary_crossentropy: 0.1831 - val_loss: 0.9072 - val_binary_crossentropy: 0.8799
Epoch 98/100
529/529 [==============================] - 79s 150ms/step - loss: 0.2094 - binary_crossentropy: 0.1823 - val_loss: 0.9324 - val_binary_crossentropy: 0.9051
Epoch 99/100
529/529 [==============================] - 79s 149ms/step - loss: 0.2094 - binary_crossentropy: 0.1824 - val_loss: 0.9347 - val_binary_crossentropy: 0.9075
Epoch 100/100
529/529 [==============================] - 80s 152ms/step - loss: 0.2100 - binary_crossentropy: 0.1831 - val_loss: 0.8904 - val_binary_crossentropy: 0.8632


In [16]:
model.save('WDL_bert.h5')

In [15]:
# history = model.fit(train_model_input, train[['lable']].values,
#                     batch_size=256, epochs=100, verbose=2, validation_split=0.2, )
from keras.callbacks import *
# 回调函数
file_path = "deepfm_bert.h5"
earlystopping = EarlyStopping(monitor="val_loss", patience=3)
checkpoint = ModelCheckpoint(
    file_path, save_weights_only=False, verbose=1, save_best_only=True)
callbacks_list = [earlystopping, checkpoint]

history = model.fit(train_model_input, train[['lable']].values, 
                   batch_size=256,
                  epochs=100,
                  validation_split=0.2,
                  callbacks=callbacks_list,
                  shuffle=True)
# history=model.fit(train_sparse_x+train_dense_x,
#                   train_label, 
#                   epochs=100,
#                   batch_size=32,
#                   validation_data=(valid_sparse_x+valid_dense_x, valid_label))

Epoch 1/100
2114/2114 [==============================] - 322s 149ms/step - loss: 0.4654 - binary_crossentropy: 0.4602 - val_loss: 0.4451 - val_binary_crossentropy: 0.4363

Epoch 00001: val_loss improved from inf to 0.44514, saving model to deepfm_bert.h5


D:\Anaconda3 2020.11\envs\tf-gpu-25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
2114/2114 [==============================] - 309s 146ms/step - loss: 0.4305 - binary_crossentropy: 0.4178 - val_loss: 0.4479 - val_binary_crossentropy: 0.4337

Epoch 00002: val_loss did not improve from 0.44514
Epoch 3/100
2114/2114 [==============================] - 310s 147ms/step - loss: 0.4079 - binary_crossentropy: 0.3916 - val_loss: 0.4685 - val_binary_crossentropy: 0.4509

Epoch 00003: val_loss did not improve from 0.44514
Epoch 4/100
2114/2114 [==============================] - 309s 146ms/step - loss: 0.3873 - binary_crossentropy: 0.3683 - val_loss: 0.5188 - val_binary_crossentropy: 0.4990

Epoch 00004: val_loss did not improve from 0.44514


In [ ]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [15]:
model.save('WDL_bert.h5')

D:\Anaconda3 2020.11\envs\tf-gpu-25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
